Desejamos analisar o comportamento de usuários de uma plataforma a partir dos dados "tracking.csv".

Queremos que o usuário entre no site e compre o ingresso de um evento de IA, então é interessante analisar o comportamento dos usuários.

In [2]:
import pandas as pd

data = pd.read_csv('Dados/tracking.csv', sep=',')
data.head()

,inicial,palestras,contato,comprou,patrocinio
0,1,1,0,0,0
1,1,1,0,0,0
2,1,1,0,0,0
3,1,1,0,0,0
4,1,1,0,0,0


Queremos estimar a propriedade "comprou", pois queremos prever se o usuário irá comprar ou não o ingresso, baseado nas outras características dos dados.

**Então nossa próxima fase é separar os dados de entrada (variáveis independentes) dos nossos rótulos/resultados, que são os nossos resultados esperados (variáveis dependentes)**

In [3]:
X = data.drop('comprou', axis=1)
X.head() # Meus dados sem a coluna "comprou" (pois não podemos dar acesso aos rótulos, que seria a coluna comprou)

,inicial,palestras,contato,patrocinio
0,1,1,0,0
1,1,1,0,0
2,1,1,0,0
3,1,1,0,0
4,1,1,0,0


In [4]:
y = data['comprou']
y.head()

0    0
1    0
2    0
3    0
4    0
Name: comprou, dtype: int64

Agora iremos separar o nosso dataset em dados de treino e teste com a função *train_test_split*.

In [5]:
from sklearn.model_selection import train_test_split

# Iremos separar o nosso dataset em treino (80%) e teste (20%) com a seed 42
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

Seria interessante validarmos se os rótulos de treino e testes estão bem distribuídos para que nosso modelo não seja treinado de forma enviesada.

Fazemos isto verificando a quantidade de cada valor único nos nossos rótulos e verificar se estão bem distribuidos.

In [6]:
# Verificando a quantidade de cada rótulo nos dados de treino
y_train.value_counts()

comprou
0    53
1    26
Name: count, dtype: int64

In [7]:
# Verificando a quantidade de cada rótulo nos dados de teste
y_test.value_counts()

comprou
0    13
1     7
Name: count, dtype: int64

É possível observar que os dados não estão muito bem distribuidos entre si.

Porém, podemos utilizar um parametro da função *train_test_split* chamado *stratify* para distribuir de forma **proporcional** nossos dados em relação aos nossos **rótulos**.

**O que é a Estratificação?** 

Em problemas de classificação, essa técnica garante que as proporções das diferentes classes sejam mantidas nos conjuntos de treino e teste. Esse método é especialmente importante em problemas de classificação com classes desbalanceadas.

Manter as proporções das classes ajuda a garantir que o modelo seja treinado e avaliado de forma justa, evitando cenários onde, por exemplo, o modelo "vê" muito mais exemplos de uma classe do que da outra.

In [8]:
# O parametro stratify=y distribui os dados de forma proporcional em relação aos rótulos
# Porém, devemos remover o parametro test_size, do contrário, a estratificação não irá funcionar
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, stratify=y)

In [9]:
y_train.value_counts()

comprou
0    49
1    25
Name: count, dtype: int64

In [10]:
y_test.value_counts()

comprou
0    17
1     8
Name: count, dtype: int64

Após separar o dataset, iremos treinar o nosso modelo utilizando um algoritmo chamado "Support Vector Machine" (SVM) para resolução de problemas lineares (LinearSVC).

In [11]:
from sklearn.svm import LinearSVC

model = LinearSVC()

# Treinamos o modelo com os dados e rótulos de treino
model.fit(X_train, y_train)

LinearSVC()

Após o modelo treinado, iremos realizar o procedimento de testes, onde iremos passar para o modelo prever os dados de teste.

In [12]:
model_predict = model.predict(X_test)

Os resultados da predição do modelo foram armazenados na variável *model_predict*.

Com estes resultados, agora iremos realizar o procedimento de avaliação do nosso modelo com algumas métricas conhecidas:

- Acurácia
- Recall (Revocação/Sensibilidade)
- Precisão
- F1 Score

In [13]:
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score

# 1° Parametro: Rótulos de teste esperados (verdadeiros)
# 2° Parametro: Rótulos previstos pelo modelo
accuracy = accuracy_score(y_true=y_test, y_pred=model_predict)
print(f'Acurácia: {accuracy*100:.2f}%')

Acurácia: 96.00%


In [14]:
# Recall -> Não posso errar na minha negação
recall = recall_score(y_test, model_predict)
print(f'Recall: {recall*100:.2f}%')

Recall: 87.50%


In [15]:
# Precisão -> Não posso errar na minha afirmação
precision = precision_score(y_test, model_predict)
print(f'Precisão: {precision*100:.2f}%')

Precisão: 100.00%


In [16]:
# F1_Score -> Não posso errar na minha afirmação ou negação (média harmônica, equilibrio entre Precisão e Recall)
f1_score = f1_score(y_test, model_predict)
print(f'F1 Score: {f1_score*100:.2f}%')

F1 Score: 93.33%


**Pontos importantes para avaliar o nosso modelo em questão:**

- Devemos sempre identificar a pessoa que não tem interesse em comprar para exibir alguma mensagem.
- Um Falso Positivo, ou seja, o modelo afirmar que a pessoa irá comprar um ingresso, porém na realidade ela não comprar, poderá gerar consequências negativas ao negócio.

Portanto, a métrica ideal para avaliação do nosso modelo seria o de precisão, pois ele foca nos erros de *Falsos Positivos*, para que o modelo sempre evite errar na afirmação se o usuário irá comprar.

**Conclusão:**

Para avaliar um modelo de Machine Learning nós devemos realizar a seguinte pergunta:
- Quais erros o meu modelo **NÃO PODE** cometer?
    - *Falsos Negativos* ou *Falsos Positivos*?

Descobrindo qual erro deve ser evitado, deveremos focar nossa avaliação na métrica de *Precisão* ou *Recall*.

Se ambos os erros não podem ocorrer, é necessário focar a avaliação na métrica de *F1 Score*, que seria a métrica de média harmônica entre *Recall* e *Precisão*.